## DNN model for MNIST dataset

### Prepare Models

In [1]:
import numpy as np
import torch
import tensorflow as tf

device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-02-02 14:40:14.492325: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 14:40:14.524993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 14:40:14.525018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 14:40:14.525678: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 14:40:14.530360: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Convert to PyTorch format [batch_size, total pixels]
# Since images are already normalized and flattened for TensorFlow, we can use the same arrays
train_images_pt = torch.tensor(train_images_tf).float()
test_images_pt = torch.tensor(test_images_tf).float()
train_labels_pt = torch.tensor(train_labels)
test_labels_pt = torch.tensor(test_labels)

In [3]:
# resize

train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

2024-02-02 14:40:21.494027: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-02 14:40:21.494664: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-02 14:40:21.494721: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
#resize
import torch.nn.functional as F

# Assuming test_images_pt is your PyTorch tensor with shape [num_samples, 784]
test_images_pt_reshaped = test_images_pt.view(-1, 1, 28, 28)  # Reshape to [num_samples, channels, height, width]

# Downsample images
test_images_pt_downsampled = F.interpolate(test_images_pt_reshaped, size=(14, 14), mode='bilinear', align_corners=False)

# Flatten the images back to [num_samples, 14*14]
test_images_pt = test_images_pt_downsampled.view(-1, 14*14)


### Train Model

#### Input-Dense-Dense 3_layer model

In [127]:
from tensorflow import keras
num_classes = 10
input_size = 14 * 14
layer_1 = 25

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(input_size,)),  # Adjusted for 28x28 images
    keras.layers.Dense(layer_1, activation = 'relu'),                         # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model_tf.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 25)                4925      
                                                                 
 dense_5 (Dense)             (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=20, batch_size=256, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/20


2024-02-02 14:40:35.840659: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-02 14:40:35.881460: I external/local_xla/xla/service/service.cc:168] XLA service 0x561a2de78a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-02 14:40:35.881474: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-02-02 14:40:35.884579: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-02 14:40:35.892674: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1706856035.939233 1037340 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


211/211 [==============================] - 1s 1ms/step - loss: 1.3213 - accuracy: 0.6456 - val_loss: 0.5634 - val_accuracy: 0.8847
Epoch 2/20
211/211 [==============================] - 0s 1ms/step - loss: 0.4871 - accuracy: 0.8784 - val_loss: 0.3311 - val_accuracy: 0.9205
Epoch 3/20
211/211 [==============================] - 0s 1ms/step - loss: 0.3598 - accuracy: 0.9029 - val_loss: 0.2686 - val_accuracy: 0.9320
Epoch 4/20
211/211 [==============================] - 0s 1ms/step - loss: 0.3106 - accuracy: 0.9140 - val_loss: 0.2390 - val_accuracy: 0.9362
Epoch 5/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2822 - accuracy: 0.9210 - val_loss: 0.2193 - val_accuracy: 0.9410
Epoch 6/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2619 - accuracy: 0.9263 - val_loss: 0.2073 - val_accuracy: 0.9432
Epoch 7/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2471 - accuracy: 0.9299 - val_loss: 0.1959 - val_accuracy: 0.9467
Epoch 8/20
211/21

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, layer_1)  # Flatten 28*28 and feed into 56 neurons
        self.fc2 = nn.Linear(layer_1, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)
    
model_pt = Net()

In [17]:
# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[1].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

#### Input-Dense-Dense-Dense 4_layers DNN Model

In [128]:
from tensorflow import keras
num_classes = 10
input_size = 14 * 14
layer_1 = 24
layer_2 = 14

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(input_size,)),  # Adjusted for 28x28 images
    keras.layers.Dense(layer_1, activation = 'relu'),   
    keras.layers.Dense(layer_2, activation = 'relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model_tf.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                4728      
                                                                 
 dense_7 (Dense)             (None, 14)                350       
                                                                 
 dense_8 (Dense)             (None, 10)                150       
                                                                 
Total params: 5228 (20.42 KB)
Trainable params: 5228 (20.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [129]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=20, batch_size=256, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/20
211/211 [==============================] - 1s 1ms/step - loss: 1.3609 - accuracy: 0.5878 - val_loss: 0.5127 - val_accuracy: 0.8785
Epoch 2/20
211/211 [==============================] - 0s 1ms/step - loss: 0.4517 - accuracy: 0.8743 - val_loss: 0.3067 - val_accuracy: 0.9163
Epoch 3/20
211/211 [==============================] - 0s 1ms/step - loss: 0.3438 - accuracy: 0.9005 - val_loss: 0.2544 - val_accuracy: 0.9305
Epoch 4/20
211/211 [==============================] - 0s 1ms/step - loss: 0.3016 - accuracy: 0.9130 - val_loss: 0.2260 - val_accuracy: 0.9380
Epoch 5/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2740 - accuracy: 0.9211 - val_loss: 0.2048 - val_accuracy: 0.9422
Epoch 6/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2530 - accuracy: 0.9277 - val_loss: 0.1912 - val_accuracy: 0.9442
Epoch 7/20
211/211 [==============================] - 0s 1ms/step - loss: 0.2362 - accuracy: 0.9312 - val_loss: 0.1796 - val_accuracy: 0.9480
Epoch 

In [131]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, layer_1)  # Flatten 28*28 and feed into 56 neurons
        self.fc2 = nn.Linear(layer_1, layer_2)
        self.fc3 = nn.Linear(layer_2, num_classes) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim = 1)
    
model_pt = Net()

In [132]:
# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[1].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

weights, biases = model_tf.layers[2].get_weights()
model_pt.fc3.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc3.bias = nn.Parameter(torch.from_numpy(biases))

### Test Converted model_pt

In [133]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images_tf[189:190]  # Reshape to (1, 784) for DNN
controlled_input_pt = test_images_pt[189:190]

# Test TensorFlow Model
output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)

# Test PyTorch Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)
print("PyTorch Basic Model Output:", output_pt)

1/1 [==============================] - 0s 25ms/step
TensorFlow Basic Model Output: [[1.4794455e-04 9.7048104e-01 1.1314460e-03 8.0442689e-03 5.0906963e-03
  7.1880864e-03 3.6779244e-03 1.4568124e-03 1.5915688e-03 1.1902171e-03]]
PyTorch Basic Model Output: tensor([[1.4794e-04, 9.7048e-01, 1.1314e-03, 8.0443e-03, 5.0907e-03, 7.1881e-03,
         3.6779e-03, 1.4568e-03, 1.5916e-03, 1.1902e-03]])


In [134]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset for test data
test_dataset = TensorDataset(test_images_pt, test_labels_pt)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

def evaluate_pytorch_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_loader)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')


Accuracy of the PyTorch model on the test images: 95.56000000%


In [135]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=256):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [136]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images_tf)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/1 [==============================] - 0s 13ms/step
Number of mismatches: 0 out of 10000 samples


### Orion Specialized q_aware_model


In [172]:
import tensorflow_model_optimization as tfmot

# Apply quantization to the layers
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model_tf)
#q_aware_model.set_weights(model_tf.get_weights())

# 'quantize_model' requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_6 (Quantize  (None, 196)               3         
 Layer)                                                          
                                                                 
 quant_dense_6 (QuantizeWra  (None, 24)                4733      
 pperV2)                                                         
                                                                 
 quant_dense_7 (QuantizeWra  (None, 14)                355       
 pperV2)                                                         
                                                                 
 quant_dense_8 (QuantizeWra  (None, 10)                155       
 pperV2)                                                         
                                                                 
Total params: 5246 (20.49 KB)
Trainable params: 5228 (

In [173]:
batch_size = 256
epochs = 1
history = q_aware_model.fit(train_images_tf, train_labels,
                            epochs=epochs,
                            validation_split=0.1)

# Evaluate the model
test_loss, test_acc = q_aware_model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

1688/1688 [==============================] - 3s 2ms/step - loss: 0.1575 - accuracy: 0.9532 - val_loss: 0.1227 - val_accuracy: 0.9643
313/313 - 0s - loss: 0.1431 - accuracy: 0.9582 - 227ms/epoch - 726us/step

Test accuracy: 0.9581999778747559


In [174]:
# Generate predictions
q_predictions_tf = get_predictions_tf(q_aware_model, test_images_tf)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(q_predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/8 [==>...........................] - ETA: 0s

1/1 [==============================] - 0s 41ms/step
Number of mismatches: 138 out of 10000 samples


In [175]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [np.array(train_images[i:i+1], dtype=np.float32)]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
q_aware_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmprjafcazd/assets


INFO:tensorflow:Assets written to: /tmp/tmprjafcazd/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-02-02 18:30:06.052654: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-02 18:30:06.052673: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-02 18:30:06.052761: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprjafcazd
2024-02-02 18:30:06.053851: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-02 18:30:06.053858: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmprjafcazd
2024-02-02 18:30:06.056923: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
20

#### Save Models for 196_25_10

In [34]:
import os
# Tensorflow
arch_folder = "./input-dense-dense/"
os.makedirs(arch_folder, exist_ok=True)

model_name = "196_25_10"
model_tf.save(arch_folder + model_name + '.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open(arch_folder + model_name + '.tflite', 'wb') as f:
    f.write(tflite_model)


/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /tmp/tmpgdyu27rf/assets


INFO:tensorflow:Assets written to: /tmp/tmpgdyu27rf/assets
2024-02-02 14:56:06.902758: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-02 14:56:06.902775: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-02 14:56:06.903007: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpgdyu27rf
2024-02-02 14:56:06.903629: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-02 14:56:06.903639: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpgdyu27rf
2024-02-02 14:56:06.905303: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-02 14:56:06.905787: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-02 14:56:06.932669: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [126]:
# Specialized Model

# kerase2cairo
q_aware_model.save(arch_folder + "q_aware_" + model_name + ".h5")
open(arch_folder + "q_aware_"+model_name + ".tflite", "wb").write(q_aware_tflite_model)

/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7456

In [35]:
# pytorch

# Save entire model
torch.save(model_pt, arch_folder + model_name + ".pt")
# Save only the state_dict
torch.save(model_pt.state_dict(), arch_folder + model_name + ".pth")
with torch.no_grad():
    torch.onnx.export(model_pt, controlled_input_pt, arch_folder + model_name + ".onnx")

#### Save Model for 196_24_14_10

In [176]:
import os
# Tensorflow
arch_folder = "./input-dense-dense-dense/"
os.makedirs(arch_folder, exist_ok=True)

model_name = "196_24_14_10"
model_tf.save(arch_folder + model_name + '.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open(arch_folder + model_name + '.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp2xuk24hy/assets


/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
INFO:tensorflow:Assets written to: /tmp/tmp2xuk24hy/assets
2024-02-02 18:31:49.180569: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-02 18:31:49.180588: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-02 18:31:49.180678: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp2xuk24hy
2024-02-02 18:31:49.181064: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-02 18:31:49.181070: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp2x

In [177]:
# Specialized Model

# kerase2cairo
q_aware_model.save(arch_folder + "q_aware_" + model_name + ".h5")
open(arch_folder + "q_aware_"+model_name + ".tflite", "wb").write(q_aware_tflite_model)

/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8200

In [ ]:
# pytorch

# Save entire model
torch.save(model_pt, arch_folder + model_name + ".pt")
# Save only the state_dict
torch.save(model_pt.state_dict(), arch_folder + model_name + ".pth")
with torch.no_grad():   
    torch.onnx.export(model_pt, controlled_input_pt, arch_folder + model_name + ".onnx")